# Data Preparation

In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import math
from utils import * # utils.py contains all etl functions needed for data preparation
import pickle
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Data Extraction

In [2]:
# data file locations
FILE_PATH = '0_raw_data/'
FILE_EXTENSION = '.csv'
INPUT_DATA_DIR = '1_input/input.csv'
STARTING_TABLES  = \
['pt_icu_outcome', 'vitals_hourly', 'labs_hourly', 
 'gcs_hourly', 'admissions', 'patients']
FILE_DIR = [FILE_PATH + name + FILE_EXTENSION for name in STARTING_TABLES]

# data ingestion
raw_icu = pd.read_csv(FILE_DIR[0])
raw_vitals = pd.read_csv(FILE_DIR[1])
raw_labs = pd.read_csv(FILE_DIR[2])
raw_gcs = pd.read_csv(FILE_DIR[3]) #Glasgow Coma Score
raw_ad = pd.read_csv(FILE_DIR[4])
raw_patients = pd.read_csv(FILE_DIR[5])

## Data Wrangling

In [3]:
# Data Manipulation
# pt_icu_outcome ETL
pt_icu = pt_icu_etl(raw_icu)
pt_icu_cleaned = pd.get_dummies(pt_icu, 
                                columns=['age_bins','intime_weekday']) # --for ML model
# patients ETL (gender feature keep 'M')
patients_cleaned = pd.get_dummies(raw_patients.loc[:,('subject_id','gender')],
                                  drop_first=True,).set_index('subject_id') #--for ML model
# admission ETL
admission = admission_etl_function(raw_ad)
# --for ML model
adm_dummies = pd.get_dummies(admission, drop_first=True,
                             columns = ['re_adm_in30d','english_speaker'])
adm_dummies = pd.get_dummies(adm_dummies, 
                             columns = ['insurance','admission_type'],
                             prefix=['insure','adm_type'])
cols_to_keep_admission = ('hadm_id','len_of_adm','re_adm_in30d_True',
                          'english_speaker_True', 'insure_government','insure_medicaid',
                          'insure_medicare','insure_private','insure_selfpay',
                          'adm_type_elective','adm_type_emergency','adm_type_urgent')
admission_cleaned = adm_dummies.reset_index().loc[:, cols_to_keep_admission]

# vitals, labs and gcs hourly tables ETL
first_24_vital_agg = hourly_vitals_etl(raw_vitals)
first_24_labs_agg = hourly_labs_etl(raw_labs)
first_24_gcs_agg = hourly_gcs_etl(raw_gcs)

In [14]:
# merge pt_icu_outcome with admission, vitals, GCSs and labs tables
# for Research Question 1
icu_n_patients = pd.merge(pt_icu_cleaned, 
                           patients_cleaned, 
                           how='inner', on='subject_id', suffixes=('','_pt'))
icu_n_admission = pd.merge(icu_n_patients, 
                           admission_cleaned, 
                           how='inner', on='hadm_id', suffixes=('','_adm'))
icu_n_vital = pd.merge(icu_n_admission, 
                       first_24_vital_agg, 
                       how='inner', on='icustay_id', suffixes=('','_vt'))
icu_n_gcs = pd.merge(icu_n_vital, 
                     first_24_gcs_agg, 
                     how='inner', on='icustay_id', suffixes=('','_gcs'))
icu_n_labs = pd.merge(icu_n_gcs, 
                      first_24_labs_agg, 
                      how='inner', on='icustay_id', suffixes=('','_labs'))

# data for research question 2
rq2_dataset = pd.merge(pt_icu,
                       admission,
                       how='inner', on='hadm_id', suffixes=('','_hadm'))

# data for visualisation
pre_OHE_merge0 = pd.merge(pt_icu,
                          raw_patients,
                          how='inner', on='subject_id', suffixes=('','_pt'))
pre_OHE_merge1 = pd.merge(pre_OHE_merge0, 
                          admission, 
                          how='inner', on='hadm_id', suffixes=('','_adm'))
pre_OHE_merge2 = pd.merge(pre_OHE_merge1,
                          first_24_vital_agg, 
                          how='inner', on='icustay_id', suffixes=('','_vital'))
pre_OHE_merge3 = pd.merge(pre_OHE_merge2, 
                          first_24_gcs_agg, 
                          how='inner', on='icustay_id', suffixes=('','_gcs'))
pre_OHE_merge4 = pd.merge(pre_OHE_merge3, 
                          first_24_labs_agg, 
                          how='inner', on='icustay_id', suffixes=('','_labs'))

In [15]:
rq2_dataset.columns

Index(['row_id', 'subject_id', 'dob', 'hadm_id', 'admittime', 'dischtime',
       'icustay_id', 'age_years', 'intime', 'outtime', 'los', 'hosp_deathtime',
       'icu_expire_flag', 'hospital_expire_flag', 'dod', 'expire_flag',
       'ttd_days', 'age_bins', 'intime_weekday', 'icu_adm_weekend', 'ttd_bins',
       'standard_mortality_label', 'row_id_hadm', 'admittime_hadm',
       'dischtime_hadm', 'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag_hadm', 'has_chartevents_data', 'prev_dischtime',
       'tt_next_adm_days', 're_adm_in30d', 'len_of_adm', 'english_speaker'],
      dtype='object')

In [17]:
# drop columns 
# - drop columns with only 1 unique value
col_w_1_value = list((icu_n_labs.nunique())[icu_n_labs.nunique()<2].index)
icu_n_labs_0 = icu_n_labs.drop(col_w_1_value, axis=1)
# - drop columns that are not needed for ML model development
cols_to_drop_rq1 = \
['row_id', 'subject_id', 'dob', 'hadm_id', 'admittime', 'dischtime',
 'icustay_id', 'age_years', 'intime', 'outtime', 'hosp_deathtime',
 'icu_expire_flag', 'hospital_expire_flag', 'dod', 'expire_flag',
 'ttd_days', 'icu_adm_weekend', 'ttd_bins', ] # columns not needed
# - keep columns needed for research question 2
cols_to_keep_rq2 = \
['subject_id', 'hadm_id', 'icustay_id', 'age_years','age_bins', 'icu_adm_weekend',
 'standard_mortality_label', 'los', 're_adm_in30d',
 'insurance', 'admission_type',] # columns needed
# - drop columns that are not needed for data visualisation
cols_to_drop_viz = \
['row_id_pt', 'dob_pt', 'dod_pt', 'expire_flag_pt', 
 'row_id_adm', 'admittime_adm', 'dischtime_adm','hospital_expire_flag_adm',] # columns not needed

# save data tables
pt_icu_final = icu_n_labs_0.drop(cols_to_drop_rq1, axis=1)
pt_icu_final.to_pickle("1_input/input_rq1.pkl") # for research question 1

rq2_final = rq2_dataset.loc[:,cols_to_keep_rq2] # csv to be read in R
rq2_final.to_csv("1_input/input_rq2.csv") #for research question 2

viz_final = pre_OHE_merge4.drop(cols_to_drop_viz, axis=1)
viz_final.to_pickle("1_input/input_viz.pkl") #for research question 2

In [18]:
dataset_brief(icu_n_labs_0,'final table for research q1 before dropping id')

Dataset final table for research q1 before dropping id provided of size:	51065 rows and 80 columns

List of columns in the dataset:
['row_id', 'subject_id', 'dob', 'hadm_id', 'admittime', 'dischtime', 'icustay_id', 'age_years', 'intime', 'outtime', 'los', 'hosp_deathtime', 'icu_expire_flag', 'hospital_expire_flag', 'dod', 'expire_flag', 'ttd_days', 'icu_adm_weekend', 'ttd_bins', 'standard_mortality_label', 'age_bins_under44', 'age_bins_45-54', 'age_bins_55-64', 'age_bins_65-74', 'age_bins_over75', 'intime_weekday_Friday', 'intime_weekday_Monday', 'intime_weekday_Saturday', 'intime_weekday_Sunday', 'intime_weekday_Thursday', 'intime_weekday_Tuesday', 'intime_weekday_Wednesday', 'gender_M', 'len_of_adm', 're_adm_in30d_True', 'english_speaker_True', 'insure_government', 'insure_medicaid', 'insure_medicare', 'insure_private', 'insure_selfpay', 'adm_type_elective', 'adm_type_emergency', 'adm_type_urgent', 'bp_elevated', 'bp_hbp_s1', 'bp_hbp_s2', 'bp_hyptsn_crisis', 'abnorm_spo2', 'fever', '

In [19]:
icu_n_labs_0.standard_mortality_label.value_counts()

False    47368
True      3697
Name: standard_mortality_label, dtype: int64

In [20]:
dataset_brief(rq2_final,'final table for research q2 before dropping id')

Dataset final table for research q2 before dropping id provided of size:	61516 rows and 11 columns

List of columns in the dataset:
['subject_id', 'hadm_id', 'icustay_id', 'age_years', 'age_bins', 'icu_adm_weekend', 'standard_mortality_label', 'los', 're_adm_in30d', 'insurance', 'admission_type']
over 50% values missing in following columns:
Series([], dtype: float64)
Column subject_id has 46464 unique values
Column hadm_id has 57771 unique values
Column icustay_id has 61516 unique values
